# Forecast

Based on our current scouting, as well as The Blue Alliance's OPR score, and any other criteria we have, guess the outcome of the matches to be played, and display the matches
that have already been played using the same criteria

*run all Data notebooks before any of the analysis notebooks.*

In [5]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys


output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Forecast_Analysis')

Loading BokehJS ...

In [6]:
# Without index_col=0, pandas will create a new, made up index for each line read from the csv.
tba_matches = pd.read_csv('tba_matches.csv',index_col=0)
tba_oprs = pd.read_csv('tba_oprs.csv',index_col=0)
app_event_summary = pd.read_csv('app_event_summary.csv',index_col=0)

teams=tba_oprs.merge(app_event_summary,on='team',how="left")
teams.set_index('team',inplace=True)

tba_matches['oprs_forecast_red']=tba_matches.apply(lambda row: teams.at[row.red1,'oprs']+teams.at[row.red2,'oprs']+teams.at[row.red3,'oprs'], axis = 1 )
tba_matches['oprs_forecast_blue']=tba_matches.apply(lambda row: teams.at[row.blue1,'oprs']+teams.at[row.blue2,'oprs']+teams.at[row.blue3,'oprs'], axis = 1 )
tba_matches['scouting_forecast_red']=tba_matches.apply(lambda row: teams.at[row.red1,'scouting_points']+teams.at[row.red2,'scouting_points']+teams.at[row.red3,'scouting_points'], axis = 1 )
tba_matches['scouting_forecast_blue']=tba_matches.apply(lambda row: teams.at[row.blue1,'scouting_points']+teams.at[row.blue2,'scouting_points']+teams.at[row.blue3,'scouting_points'], axis = 1 )
# Remove all columns outside of what we are using to put in the table.
tba_matches=tba_matches[['match_number','post_result_time','red1','red2','red3','blue1','blue2','blue3',
                         'red_points','blue_points','oprs_forecast_red','oprs_forecast_blue','scouting_forecast_red','scouting_forecast_blue']]

# This rounds across all columns and rows in the dataframe, and then sorts the dataframe to ensure it is in match order.
tba_matches=tba_matches.round()
tba_matches.sort_values(by='match_number',inplace=True)


In [7]:
source = ColumnDataSource(tba_matches)

columns = [
        TableColumn(field="match_number", title="Match"),
        TableColumn(field="red1", title="Red 1"),
        TableColumn(field="red2", title="Red 2"),
        TableColumn(field="red3", title="Red 3"),
        TableColumn(field="blue1", title="Blue 1"),
        TableColumn(field="blue2", title="Blue 2"),
        TableColumn(field="blue3", title="Blue 3"),
        TableColumn(field="red_points", title="Red Actual"),
        TableColumn(field="blue_points", title="Blue Actual"),
        TableColumn(field="oprs_forecast_red", title="OPRS Red"),
        TableColumn(field="oprs_forecast_blue", title="OPRS Blue"),
        TableColumn(field="scouting_forecast_red", title="Scout Red"),
        TableColumn(field="scouting_forecast_blue", title="Scout Blue")]
    
    
    
forecast_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        fit_columns=True,
        reorderable=True)

# Forecasting 

The table shows all of the current qualifying matches for the event.  If the event has already occurred, it will show the Actual score for both Red and Blue teams.

In addition it will also show two forecasts for each match.  One is based on OPRS from The Blue Alliance, and the other is average points based on the scouting app input.




In [8]:
show(forecast_table)